In [ ]:
# -*- coding: utf-8 -*-
"""
HZR Model: Consul(pl)ants, Jumpstarts(Zombies) and the Engaged.
The below script has been written with the purpose of modeling the rates at which Consultants, Jumpstarts and Engaged 
change over the span of a Professional Services "Zombie" apocalypse. These rates of change have been defined by the 3 ODEs given in the brief
and defined in the function "f" below. None of the rates are based on actual extractions but are used for the purpose of demonstration. 
We could easily inject more meaningful measures of these rates by leveraging the Looker API. 
This script will also allow the inital values for each of these classes to be changed and therefore give the user the oppertunity to observe 
how changing these initial values will effect each class's outcome during the Professional Services Apocalypse. 
This model is a basic model which consists of only 3 classes: 
Consultants, Completed JS and Active Jumpstarts which are governed by the 3 ODEs
    Consultants'=B*Hi-d*Hi-Bs*Hi*Zi # Formula for rate of change of Consultants
    Active Jumpstarts'=Bs*Hi*Zi+g*Ri-a*Hi*Zi # Formula for rate of change of Jumpstarts
    Completed Jumpstarts'=d*Hi+a*Hi*Zi-g*Ri # Formula for rate of change of Removed
    
This Script uses Scipy's ODEint function in order to solve the given equations with given initial values.
The values on lines below can be edited in order to model a different beginning case(e.g. more or less zombies) 
"""
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import numpy as np
import psycopg2 as psql

#Set parameters. Rates are currently subjective and loosely approximated. We can solidify these by extracting 
#legitimate measures via Looker's API. 
B=0.0005#(sigma) The number of Consultants hired (per 1000 members per unit time).
d=0.000001#(delta) The natural attrition rate of Consultants (per 1000 members per unit time).
Bs=0.00004#(beta) The rate at which JS are assigned to Consultants (per 1000 members per unit time).
nd=0.0001#(phi) The rate at which a new deal is signed
g=0.00002#(gamma) The rate at which the completed class (completed JS) get reassigned as available consultant slots. (per 1000 members per unit time).
a=0.00001#(alpha) The rate at which Consultants complete Jumpstarts (per 1000 members per unit time).


#Defining function to implement given formula. y is an array made up of the initial values for each class as defined below
def rates_of_change(y, t):
    #Creates an array 'y' to store ODEs
    Hi=y[0]
    Zi=y[1]
    Ri=y[2]
    Ji=y[3]
    
    #Defining given formula from brief
    f0=B*Hi-d*Hi-Bs*Hi*Zi+g*Ri # Formula for rate of change of available Consultant slots
    f1=Bs*Hi*Zi+nd*Zi-a*Hi*Zi # Formula for rate of change of Active Jumpstarts (assigned)
    f2=d*Hi+a*Hi*Zi-g*Ri # Formula for rate of change of Completed JS (engaged Consultant slots )
    f3=nd*Ji-a*Zi # Formula for increase in Jumpstarts received by PS (whether assigned or not)
    
     #Returns each formula in an array 'y'
    return[f0,f1,f2,f3]    

#Initial values for each class
Z0=1 #Active Jumpstarts at time 0 in thousands

H0=300-Z0 #Consultants (slots of availability, 20 consultants, 15 slots each) at time 0 in thousands
R0=0 #Completed at time 0 in thousands
J0=1 #Total Jumpstarts at time 0 in thousands

#Assigning inital values to y0 array
y0=[H0,Z0,R0,J0]

#Creating an array to define the time of simulation
t = np.linspace(0,1000,1000)

#Calls odeint function on array 'y'(defined in function rates_of_change)
#function is called over a range of time t(defined above)
ans = odeint(rates_of_change, y0, t)    

#answers are saved to corrosponding array columns
H = ans[:,0]
Z = ans[:,1]
R = ans[:,2]
J = ans[:,3]

#plotting results for all ODEs
plt.figure()
plt.plot(t, H, label='Available Consultant Slots')
plt.plot(t, Z, label='Active Jumpstarts')
plt.plot(t, R, label='Completed Jumpstarts')
# plt.plot(t, J, label='Jumpstarts Total')

plt.xlabel('Time')
plt.ylabel('Population')
plt.title('PS Apocalypse')
plt.legend(loc=0)
plt.show()

In [251]:
array_extract=[]
array_dig=[]
array = (lambda x: x for x in np.nditer(H[1]))

for y in np.nditer(H[1]):
    array_extract.append(y)

array_list_H = H.tolist()
array_list_Z = Z.tolist()
array_list_R = R.tolist()


1000


In [256]:
try:
    conn = psql.connect("dbname='postgres' user='seanlugosi' host='localhost' password='academy123'")
except:
    print "Cannot Connect"

In [264]:
cur = conn.cursor()
# cur.execute("DROP TABLE cons_zombs;")
cur.execute("CREATE TABLE IF NOT EXISTS cons_zombs (id serial PRIMARY KEY, cons real, act_js real, comp_js real);")
i = 0
b = 0
m = 0
while (i < 1000):
    test = cur.mogrify("INSERT INTO cons_zombs (cons,act_js,comp_js) VALUES (%s,%s,%s);" % (array_list_H[i],array_list_Z[i],array_list_R[i]))
    i+=1